In [1]:
# Import required libraries
import os
import copy
from datetime import datetime, timedelta

# Numerical stuff
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata
import numpy as np

# Web-related
from flask import Flask,send_from_directory 
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

def get_subset(year,lat,lon):
    
    # Get daily average temperature
    T2 = xr.open_dataset('data/dayavg_T2.nc')
    aveT = T2.sel(lat=lat, lon=lon%360, method='nearest')
    aveT = aveT.sel(time = slice(year*10000,(year+1)*10000))
    aveT = aveT.T2M
    aveT = aveT.assign_coords(time=(pd.to_datetime(np.array(aveT.time).astype(int).astype(str))))

    # Get winter and summer design temperature 
    designT = pd.read_pickle('data/designT.pkl')    
    x = designT['Long']
    y = designT['Lat']
    z1 = designT['WinterT']
    z2 = designT['SummerT']
    points = [(x,y) for x,y in zip(x,y)]
    
    winterT = griddata(points, z1, (lon, lat))
    summerT = griddata(points, z2, (lon, lat))    
    
    return winterT, summerT, aveT

In [2]:
winterT, summerT, aveT = get_subset(2017, 42,-100)
monthlyT = aveT.groupby(aveT.time.dt.month).mean()

In [34]:
%%writefile app.py

# Import required libraries
import os
import copy
from datetime import datetime, timedelta

# Numerical stuff
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata
import numpy as np

# Web-related
from flask import Flask,send_from_directory 
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

def get_subset(year,lat,lon):
    
    # Get daily average temperature
    T2 = xr.open_dataset('data/dayavg_T2.nc')
    aveT = T2.sel(lat=lat, lon=lon%360, method='nearest')
    aveT = aveT.sel(time = slice(year*10000,(year+1)*10000))
    aveT = aveT.T2M
    aveT = aveT.assign_coords(time=(pd.to_datetime(np.array(aveT.time).astype(int).astype(str))))

    # Get winter and summer design temperature 
    designT = pd.read_pickle('data/designT.pkl')    
    x = designT['Long']
    y = designT['Lat']
    
    z1 = designT['WinterT']
    z2 = designT['SummerT']
    points = [(x,y) for x,y in zip(x,y)]
    
    winterT = griddata(points, z1, (lon, lat))
    summerT = griddata(points, z2, (lon, lat))    
    
    return winterT, summerT, aveT

# Get city list
citylist = pd.read_csv('data/1000-largest-us-cities-by-population-with-geographic-coordinates.csv') 
citylist['loc'] = citylist['City'] + ', ' + citylist['State']
city_options = [{'label': city, 'value':index} for city, index in zip(citylist['loc'],citylist.index)]

elec_price = pd.read_pickle('data/UtilPrice/elec_price.pkl')
pr_price = pd.read_pickle('data/UtilPrice/pr_price.pkl')
ho_price = pd.read_pickle('data/UtilPrice/ho_price.pkl')
ng_price = pd.read_pickle('data/UtilPrice/ng_price.pkl')

app = dash.Dash(static_folder='static')
app.title = 'Magpie Home Energy'
server = app.server
year = 2017

## Bootstrap CSS
# Create global chart template & layout
app.css.append_css({'external_url': 'https://cdn.rawgit.com/plotly/dash-app-stylesheets/2d266c578d2a6e8850ebce48fdb52759b2aef506/stylesheet-oil-and-gas.css'})
mapbox_access_token = 'pk.eyJ1Ijoiam9lLW1hZ3BpZXNvbHV0aW9ucy1jbyIsImEiOiJjajdqampocnEyM2tuMzNwZHd2MTdia244In0.fVrkGAPX0qqXzET1x6xL0A'

# Not sure what this does?
if 'DYNO' in os.environ:
    app.scripts.append_script({
        'external_url': 'https://cdn.rawgit.com/chriddyp/ca0d8f02a1659981a0ea7f013a378bbd/raw/e79f3f789517deec58f41251f7dbb6bee72c44ab/plotly_ga.js'  # noqa: E501
})

# Create app layout
app.layout = html.Div(
    className='ten columns offset-by-one',
    
    # Logo & Title
    children=[html.Div(
            className='row',
            children=[html.Img(src='static/MagpieLogo.png',
                      className='two columns',
                      style={'height': '80',
                             'width': '320',
                             'float': 'left',
                             'position': 'relative'},
                     ),
             html.H1('',
                     className='eight columns',
                    )            
            ],
        ),

        html.Div(
            className = 'row',
            children=[html.Div(
                [dcc.Dropdown(id='latlon_dropdown',
                              options=city_options,
                              placeholder='Enter City',
                              value='')
                ],
                className = 'five columns',
                style={'margin-top': '20'}
                ), 
             
            html.Div(
                [html.Div(id='elec_slider_text'),
                 
                 dcc.RangeSlider(id='elec_slider',min=0, max=300,step=5,
                                 allowCross=False,
                                 marks={0: '$0',
                                        50: '$50',
                                        100: '$100',
                                        150: '$150',
                                        200: '$200',
                                        250: '$250',
                                        300: '$300'},
                                 value=[50, 120]), 

                 dcc.RadioItems(id='elecUnit',
                                options = [{'label': '$', 'value': 'dollar'},
                                           {'label': 'KWH', 'value': 'kwh'}],
                                value = 'dollar',
                                labelStyle={'display': 'inline-block'},
                                style = {'textAlign': 'center'}),
                ],
                className = 'five columns',
                style={'margin-top': '20'}
                ),
            ],   
        ),
                
        html.Div(
            className='row',
            children=[html.Div(
                className='five columns',
                style={'margin-top': '5'},
                
                children=[dcc.Graph(id = 'mapbox',style = {'height': '200'}),
                                                    
                          dcc.Graph(id='daily_average'),
                          
                          dcc.RadioItems(id = 'degUnit',
                                        options=[{'label': 'Celicius', 'value': 'C'},
                                                 {'label': 'Fahrenheit', 'value': 'F'}
                                                ],
                                        value='C',
                                        labelStyle={'display': 'inline-block'},
                                        style = {'textAlign': 'center'},
                                       ),
                         ]
            ),
                      
                      html.Div(className='five columns',
                               style={'margin-top': '20'},
                               children=[dcc.Graph(id='elec_usage'),

                                        html.Div(''),

                                        dcc.RadioItems(
                                            options=[
                                                {'label': 'Natural Gas', 'value': 'gas'},
                                                {'label': 'Heating Oil', 'value': 'oil'},
                                                {'label': 'Propane', 'value': 'propane'},
                                                {'label': 'N/A', 'value': 'na'}
                                            ],
                                            value='gas',
                                            labelStyle={'display': 'inline-block'},
                                            style = {'textAlign': 'center'},
                                        )
                                        ],
                ),
            ]
        )        
    ]
)


@server.route('/favicon.ico')
def favicon():
    return send_from_directory(os.path.join(server.root_path, 'static'),'favicon.ico',mimetype='image/vnd.microsoft.icon')


@app.callback(Output(component_id='mapbox',component_property='figure'),
    [Input(component_id='latlon_dropdown', component_property='value')])
def update_map(location):
    '''
    Function to re-draw map based on location
    '''
    if location: 
        lat = citylist.loc[location]['Lat']
        lon = citylist.loc[location]['Lon']
        
    else:
        lat = 42.36
        lon = -71.06

    return {
        'data': [
            {'type' : 'scattermapbox',
             'lat' : [lat],
             'lon' : [lon],
             'mode' : 'markers',
             'marker' : dict(size = 8),
             'text' : location}],
        
        'layout': {
            'hovermode' : 'closest',
            'hoverinfo' : 'none',
            'margin' : {'l' : 0, 'r' : 0, 't' : 0, 'b' : 0},
            'mapbox' : {
                'accesstoken' : mapbox_access_token,
                'bearing' : 0,
                'center' : {'lat' : lat, 'lon' : lon},
                'pitch' : 0,
                'style' : 'light',
                'zoom' : 8
            }
        }
    }

@app.callback(Output(component_id='daily_average',component_property='figure'),
              [Input(component_id='latlon_dropdown', component_property='value'),
              Input(component_id='degUnit', component_property='value')])
def update_temperature(location,degUnit):
    '''
    Function to update average daily temperature
    '''    
    
    # Date
    start_date = datetime(year,1,1)
    end_date = datetime(year+1,1,1)
    n_days = end_date - start_date
    date_list = [start_date + timedelta(days=x) for x in range(0, n_days.days)]
    
    if location: 
        lat = citylist.loc[location]['Lat']
        lon = citylist.loc[location]['Lon']
    else:
        lat = 42.36
        lon = -71.06
        
    winterT, summerT, aveT = get_subset(year,lat,lon)
    
    if degUnit is 'F':
        aveT = (aveT-273.15)*1.8 + 32
        winterT = winterT * 1.8 + 32
        summerT = summerT * 1.8 + 32
    else:
        aveT = aveT-273.15
    
    return {
        'data': [
            {'x': date_list, 
             'y': np.around(aveT.values,1), 
             'type': 'scatter', 
             'name': 'Daily Mean'},
                 
             {'x': date_list, 
              'y': np.round(winterT,1) * np.ones(365), 
              'type': 'scatter', 
              'name': 'Winter Design T'},

             {'x': date_list, 
              'y': np.round(summerT,1) * np.ones(365), 
              'type': 'scatter', 
              'name': 'Summer Design T'},            
        ],
        
        'layout': {
            'legend':{'orientation':'h'},
            'xaxis': {'tickformat': '%b',
                     'zeroline' : False},
#            'yaxis': {'tickformat': 'd%s'%degUnit},
            'height' : 400,
            'margin' : {'l':40, 'r':20, 't':20, 'b':20}
        }
    }


@app.callback(Output(component_id='elec_usage',component_property='figure'),
              [Input(component_id='latlon_dropdown', component_property='value'),
               Input(component_id='elec_slider', component_property='value'), 
               Input(component_id='elecUnit', component_property='value')])
def update_elec(location,elecUse,elecUnit):

    if location: 
        lat = citylist.loc[location]['Lat']
        lon = citylist.loc[location]['Lon']
    else:
        lat = 42.36
        lon = -71.06

    winterT, summerT, aveT = get_subset(year,lat,lon)

    aveT = aveT - 273.15
    HDD = 15 - aveT.where(aveT < 15, 15)
    CDD = aveT.where(aveT > 18, 18) - 18
    monthlyHDD = HDD.groupby(HDD.time.dt.month).sum()
    monthlyCDD = CDD.groupby(CDD.time.dt.month).sum()

    monthlyT = aveT.groupby(aveT.time.dt.month).mean()
    
    hottestMonth = int(np.argmax(monthlyT))
    coldestMonth = int(np.argmin(monthlyT))
    
    elecMin = elecUse[0]
    elecMax = elecUse[1]
    
    if elecUnit == 'KWH':
        elec_base = elecMin*np.ones(12)
        elec_HVAC = np.array(monthlyCDD * (elecMax - elecMin) / max(monthlyCDD))
    
    else:
        elec_base = elecMin*np.ones(12)
        elec_HVAC = np.array(monthlyCDD * (elecMax - elecMin) / max(monthlyCDD))
        
    return {
        'data': [{'x': ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                  'y': elec_base, 
                  'type': 'bar',
                  'name': 'elec_base'},
                 {'x': ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], 
                  'y': elec_HVAC, 
                  'type': 'bar',
                  'name': 'elec_HVAC'}                                                                   
                ],
        'layout': {
            'height': 300,
            'barmode': 'stack',
            'legend':{'orientation':'h'},
            'margin': dict(l = 40, r = 10, t = 20, b = 20),
            'yaxis':{'range':[0, 300],
                    'tickformat' :'$d' }
        }
    }

@app.callback(
    dash.dependencies.Output('elec_slider_text', 'children'),
    [dash.dependencies.Input('elec_slider', 'value')])
def update_output(value):
    return 'Monthly electric bill range: $%d .. $%d'%(value[0],value[1])

# Main
if __name__ == '__main__':
    app.server.run(threaded=True)

Overwriting app.py


In [35]:
!python app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jul/2018 10:54:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2018 10:54:44] "POST /_dash-update-compone

In [ ]:
import pandas as pd

In [175]:
elec_price

description,United States,New England,Connecticut,Maine,Massachusetts,New Hampshire,Rhode Island,Vermont,Middle Atlantic,New Jersey,...,New Mexico,Utah,Wyoming,Pacific Contiguous,California,Oregon,Washington,Pacific Noncontiguous,Alaska,Hawaii
Jan 2016,11.98,18.75,19.85,15.28,19.35,18.00,18.40,16.62,15.16,15.50,...,11.45,10.54,10.45,14.11,17.76,10.18,9.06,23.22,19.42,26.92
Feb 2016,12.14,19.23,20.68,15.24,19.78,18.37,18.78,16.89,15.28,15.49,...,11.26,10.61,10.62,14.13,17.69,10.35,9.21,23.35,19.81,26.77
Mar 2016,12.57,19.30,20.84,15.41,19.84,18.46,19.07,17.11,15.43,15.58,...,11.44,10.67,10.85,14.16,17.66,10.42,9.22,23.96,20.05,27.35
Apr 2016,12.43,19.82,21.15,15.65,20.65,18.67,19.43,17.52,15.72,15.52,...,11.34,10.71,10.99,11.34,12.40,10.50,9.33,24.21,20.71,26.93
May 2016,12.79,19.19,21.63,15.84,19.06,18.54,18.02,17.79,15.91,15.40,...,11.35,11.01,11.50,14.83,17.74,10.80,9.54,24.41,20.90,26.88
Jun 2016,12.72,18.90,20.94,15.90,18.52,18.13,18.69,17.57,15.98,16.05,...,12.22,11.51,11.81,15.49,18.11,10.79,9.59,25.05,21.31,27.50
Jul 2016,12.68,18.30,19.76,15.87,18.20,18.00,17.11,17.29,15.96,16.16,...,12.51,11.72,11.88,15.96,18.49,10.78,9.60,25.52,21.42,28.04
Aug 2016,12.90,18.24,18.78,16.12,18.34,18.25,18.57,17.27,16.02,16.22,...,13.02,11.89,11.75,16.38,18.88,10.86,9.57,25.08,21.07,27.45
Sep 2016,12.87,18.93,18.97,16.28,19.47,18.35,20.49,17.53,16.30,16.17,...,12.90,11.28,11.80,15.87,18.23,10.91,9.64,25.19,20.85,27.84
Oct 2016,12.46,18.76,19.93,16.28,18.71,18.93,18.89,17.78,16.16,15.29,...,12.59,10.69,11.56,12.59,13.94,10.98,9.58,24.81,21.02,27.54
